# Introduction - clustering analysis of beer review dataset

This notebook performs an in-depth clustering analysis on a dataset containing various beer characteristics, including sensory ratings, chemical properties, and user reviews. The goal is to identify meaningful groupings of beers based on different subsets of features.

### Key Steps Covered:
1. **Optimal Parameter Selection**  
   - For **KMeans**, the Elbow Method and Silhouette Score are used to determine the best number of clusters.  
   - For **Gaussian Mixture Models (GMM)**, the Bayesian Information Criterion (BIC) is used.  
   - For **DBSCAN**, the k-distance graph helps estimate an appropriate epsilon.

2. **Model Comparison**  
   - The following clustering models are tested:  
     *KMeans*, *GaussianMixture*, *DBSCAN*, *HDBSCAN*, *MeanShift*, *AgglomerativeClustering*.  
   - Each model is evaluated using:  
     - **Silhouette Score** (cluster separation)
     - **Calinski-Harabasz Score** (cluster compactness)
     - **Davies-Bouldin Score** (cluster similarity)

3. **Visualization**  
   - **PCA** is used to project high-dimensional data into 2D space for visual inspection.  
   - **Radar plots** visualize average feature values across clusters for:
     - Full feature set
     - Feature subsets: *Sensory*, *Profile*, *Chemical*, *Reviews*

4. **Export**  
   - Radar plots are saved to *projects/proj_3_team_5/plots/* for reporting and further analysis.




# Importing and data load

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, MeanShift, HDBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from kneed import KneeLocator

np.random.seed(42)

while any(marker in os.getcwd() for marker in ('exercises', 'notebooks', 'students', 'research', 'projects')):
    os.chdir("..") 
sys.path.append('.')

In [2]:
load_dotenv('projects/proj_3_team_5/.env')
df_path = os.getenv('PREPROCESSED_DATA_DIR')
df_cleaned_path = os.getenv('CLEANED_DATA_DIR')

df = pd.read_csv(df_path)
df_raw = pd.read_csv(df_cleaned_path)

# Optimal parameter selection

In [ ]:
# For KMeans, we use the Elbow method (inertia) and the Silhouette Score to select the optimal number of clusters.
# - The Elbow method helps identify the point where adding more clusters does not significantly reduce the within-cluster sum of squares (inertia), indicating a suitable number of clusters.
# - The Silhouette Score measures how similar an object is to its own cluster compared to other clusters, providing a quantitative metric for cluster quality.
# For GaussianMixture, we use the Bayesian Information Criterion (BIC) to select the optimal number of components.
# - BIC penalizes model complexity while rewarding goodness of fit, making it suitable for model selection in probabilistic clustering like GMM.
# - Lower BIC values indicate a better model, balancing fit and complexity.
inertia = []
silhouette = []
bic = []
n_range = range(2, 11)

for n in n_range:
    kmeans = KMeans(n_clusters=n, random_state=42)
    labels = kmeans.fit_predict(df)
    inertia.append(kmeans.inertia_)
    silhouette.append(silhouette_score(df, labels))
    
    gmm = GaussianMixture(n_components=n, covariance_type='full', random_state=42)
    gmm_labels = gmm.fit_predict(df)
    bic.append(gmm.bic(df))

# Plot KMeans elbow (inertia) and silhouette
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(n_range, inertia, marker='o')
plt.title('KMeans Elbow Method (Inertia)')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')

plt.subplot(1, 2, 2)
plt.plot(n_range, silhouette, marker='o')
plt.title('KMeans Silhouette Score')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.tight_layout()
plt.show()

# Plot GMM BIC
plt.figure(figsize=(6, 4))
plt.plot(n_range, bic, marker='o')
plt.title('GaussianMixture BIC')
plt.xlabel('Number of components')
plt.ylabel('BIC')
plt.tight_layout()
plt.show()


# For DBSCAN, we use the k-distance graph to estimate the optimal value of eps (the neighborhood radius).
# - The k-distance plot helps visualize the distance to the k-th nearest neighbor for each point, sorted in ascending order.
# - The "elbow" in this plot suggests a threshold where points start to become outliers, which is a good candidate for eps.

k = 7
neigh = NearestNeighbors(n_neighbors=k)
nbrs = neigh.fit(df)
distances, indices = nbrs.kneighbors(df)
k_distances = np.sort(distances[:, k-1])

plt.figure(figsize=(6, 4))
plt.plot(k_distances)
plt.title('DBSCAN k-distance Graph')
plt.xlabel('Points sorted by distance')
plt.ylabel(f'{k}th Nearest Neighbor Distance')
plt.tight_layout()
plt.show()


Based on the KMeans parameter tuning:

- The **Elbow Method** shows a visible bend around $k = 8$, suggesting diminishing returns in inertia reduction beyond this point.
- However, the **Silhouette Score** is highest at $k = 2$ (approximately $0.12$), and drops sharply for higher values of $k$, even becoming negative, which indicates poor cluster separation.

Therefore, considering both methods, the optimal number of clusters is: $k=2$


In [4]:
optimal_kmeans = 2
optimal_gmm = 2
optimal_eps = 1250

For Agglomerative Clustering, we use the same number of clusters as determined optimal for KMeans, since both are hierarchical/partitioning methods and can be compared directly.

For MeanShift and HDBSCAN, we use their default parameter estimation, as these algorithms are designed to infer the number of clusters or density structure from the data.

# PCA visualization and evaluation of clustering algorithms

In [ ]:
models = {
    'KMeans': KMeans(n_clusters=optimal_kmeans, random_state=42),
    'GaussianMixture': GaussianMixture(n_components=optimal_gmm, covariance_type='full', random_state=42),
    'DBSCAN': DBSCAN(eps=optimal_eps, min_samples=k),
    'HDBSCAN': HDBSCAN(),
    'MeanShift': MeanShift(),
    'Agglomerative': AgglomerativeClustering(n_clusters=optimal_kmeans, linkage='complete')
}

pca = PCA(n_components=2)
X_pca = pca.fit_transform(df)

metrics = {}

plt.figure(figsize=(16, 12))
for i, (name, model) in enumerate(models.items(), 1):
    try:
        labels = model.fit_predict(df)
    except Exception as e:
        labels = np.zeros(df.shape[0])
        print(f"Model {name} failed: {e}")

    plt.subplot(3, 2, i)
    plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='tab10', s=10)
    plt.title(f'{name} Clustering')
    plt.xlabel('PCA 1')
    plt.ylabel('PCA 2')
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    if n_clusters < 2:
        sil = -1
        ch = -1
        db = np.inf
    else:
        sil = silhouette_score(df, labels)
        ch = calinski_harabasz_score(df, labels)
        db = davies_bouldin_score(df, labels)
    metrics[name] = {
        'silhouette': sil,
        'calinski_harabasz': ch,
        'davies_bouldin': db,
        'n_clusters': n_clusters
    }

plt.tight_layout()
plt.show()

# Selection of best clustering model

In [ ]:
sensory_cols = ['review_aroma', 'review_appearance', 'review_palate', 'review_taste', 'review_overall']
profile_cols = ['Alcohol', 'Bitter', 'Sweet', 'Sour', 'Salty', 'Fruits', 'Hoppy', 'Spices', 'Malty', 'Astringency', 'Body']
chemical_cols = ['ABV', 'Min IBU', 'Max IBU']
review_cols = ['number_of_reviews']


metrics_df = pd.DataFrame(metrics).T
display(metrics_df)

# Choose the best clustering based on silhouette score (higher is better)
valid_metrics = metrics_df[metrics_df['n_clusters'] > 1]
if not valid_metrics.empty:
    best_model_name = valid_metrics['silhouette'].idxmax()
    print(f"Best clustering model: {best_model_name}")
else:
    best_model_name = metrics_df['silhouette'].idxmax()
    print(f"Best clustering model (by default): {best_model_name}")

best_model = models[best_model_name]
best_labels = best_model.fit_predict(df)

# Radar plot for the best clustering
radar_cols = sensory_cols + profile_cols + chemical_cols + review_cols
radar_cols = [col for col in radar_cols if col in df.columns]

# Compute mean for each cluster
cluster_means = pd.DataFrame(df[radar_cols])
cluster_means['cluster'] = best_labels
cluster_means = cluster_means.groupby('cluster').mean()

# Prepare radar plot
categories = radar_cols
N = len(categories)
angles = np.linspace(0, 2 * np.pi, N, endpoint=False).tolist()
angles += angles[:1]  # close the loop

plt.figure(figsize=(8, 8))
for idx, (cluster, row) in enumerate(cluster_means.iterrows()):
    values = row.values.flatten().tolist()
    values += values[:1]  # close the loop
    plt.polar(angles, values, label=f'Cluster {cluster}', linewidth=2)

plt.xticks(angles[:-1], categories, color='grey', size=10)
plt.title(f'Radar Plot of Cluster Means for {best_model_name}', size=15, y=1.08)
plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
plt.tight_layout()
plt.savefig(f"projects/proj_3_team_5/plots/radar_{best_model_name}_overall.png", dpi=300)
plt.show()

# Add cluster labels to the original dataframe
df_with_clusters = df_raw.copy()
df_with_clusters['cluster'] = best_labels

# Show five sample beers from each cluster only if number of clusters is smaller than 10
n_clusters = len(df_with_clusters['cluster'].unique())
if n_clusters < 10:
    print(f"\n=== Sample Beers from Each Cluster ({best_model_name}) ===")
    for cluster in sorted(df_with_clusters['cluster'].unique()):
        cluster_beers = df_with_clusters[df_with_clusters['cluster'] == cluster]
        print(f"\nCluster {cluster} ({len(cluster_beers)} beers):")
        
        # Sample 5 beers from this cluster
        sample_beers = cluster_beers.sample(n=min(5, len(cluster_beers)), random_state=42)
        display(sample_beers)
else:
    print(f"\nSkipping sample display - too many clusters ({n_clusters})")



# Clustering analysis and visualization by feature groups

In [ ]:
feature_groups = {
    'Sensory': sensory_cols,
    'Profile': profile_cols,
    'Chemical': chemical_cols,
    'Reviews': review_cols
}

for group_name, columns in feature_groups.items():
    metrics = {}
    print(f"\n=== Feature Group: {group_name} ===")
    print("Clustering results for each model:")
    X = df[columns].dropna()
    n_components = min(2, X.shape[0], X.shape[1])
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)

    plt.figure(figsize=(20, 12))
    plt.suptitle(f'Feature Group: {group_name}', fontsize=16)

    # Find optimal parameters for the models using the elbow method

    # KMeans: Find optimal number of clusters using elbow and silhouette score
    sse = []
    silhouette_scores = []
    k_range = range(2, min(11, X.shape[0]))
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(X)
        sse.append(kmeans.inertia_)
        # Only compute silhouette if more than 1 cluster
        if len(set(labels)) > 1:
            sil = silhouette_score(X, labels)
        else:
            sil = -1
        silhouette_scores.append(sil)
    # Find elbow point (simple heuristic: where the decrease sharply slows)
    if len(sse) > 2:
        elbow_k = k_range[np.argmin(np.diff(sse, 2)) + 1]
    else:
        elbow_k = k_range[0]
    # Find k with maximum silhouette score
    best_sil_k = k_range[np.argmax(silhouette_scores)]
    # Combine: pick k that is closest to elbow_k but also has high silhouette (within 90% of max)
    sil_threshold = 0.9 * max(silhouette_scores)
    candidate_ks = [k for k, sil in zip(k_range, silhouette_scores) if sil >= sil_threshold]
    if candidate_ks:
        optimal_kmeans = min(candidate_ks, key=lambda k: abs(k - elbow_k))
    else:
        optimal_kmeans = elbow_k

    # GaussianMixture: Use same optimal number of components as KMeans
    optimal_gmm = optimal_kmeans

    # DBSCAN: Find optimal eps using k-distance graph (elbow method)
    from sklearn.neighbors import NearestNeighbors
    neigh = NearestNeighbors(n_neighbors=2)
    nbrs = neigh.fit(X)
    distances, indices = nbrs.kneighbors(X)
    distances = np.sort(distances[:, 1])
    # Heuristic: take the point of maximum curvature as optimal eps

    kneedle = KneeLocator(range(len(distances)), distances, S=1.0, curve="convex", direction="increasing")
    optimal_eps = distances[kneedle.knee] if kneedle.knee is not None else np.percentile(distances, 90)
    k = 2  # min_samples

    # Agglomerative: Use same optimal number of clusters as KMeans
    optimal_agglom = optimal_kmeans

    models = {
        'KMeans': KMeans(n_clusters=optimal_kmeans, random_state=42),
        'GaussianMixture': GaussianMixture(n_components=optimal_gmm, covariance_type='full', random_state=42),
        'DBSCAN': DBSCAN(eps=optimal_eps, min_samples=k),
        'HDBSCAN': HDBSCAN(),
        'MeanShift': MeanShift(),
        'Agglomerative': AgglomerativeClustering(n_clusters=optimal_agglom, linkage='complete')
    }


    for i, (model_name, model) in enumerate(models.items(), 1):
        labels = model.fit_predict(X)

        plt.subplot(2, 3, i)
        if n_components == 1:
            plt.scatter(X_pca[:, 0], [0]*len(X_pca), c=labels, cmap='tab10', s=10)
            plt.xlabel('PCA 1')
            plt.ylabel('0 (no 2nd PCA component)')
        else:
            plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='tab10', s=10)
            plt.xlabel('PCA 1')
            plt.ylabel('PCA 2')

        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        if n_clusters < 2:
            sil = -1
            ch = -1
            db = np.inf
        else:
            sil = silhouette_score(X, labels)
            ch = calinski_harabasz_score(X, labels)
            db = davies_bouldin_score(X, labels)
    
        plt.title(f'{model_name}')
        metrics[model_name] = {
            'silhouette': sil,
            'calinski_harabasz': ch,
            'davies_bouldin': db,
            'n_clusters': n_clusters
        }

    plt.tight_layout()
    plt.show()

    print("Model metrics for this group:")
    metrics_df = pd.DataFrame(metrics).T
    display(metrics_df)

    # Plot radar plot for the best model in this group
    valid_metrics = metrics_df[metrics_df['n_clusters'] > 1]
    if not valid_metrics.empty:
        best_model_name = valid_metrics['silhouette'].idxmax()
    else:
        best_model_name = metrics_df['silhouette'].idxmax()
    print(f"Best clustering model for {group_name}: {best_model_name}")

    best_model = models[best_model_name]
    best_labels = best_model.fit_predict(X)

    # Compute mean for each cluster
    cluster_means = pd.DataFrame(X, columns=columns)
    cluster_means['cluster'] = best_labels
    cluster_means = cluster_means.groupby('cluster').mean()

    # Prepare radar plot
    categories = columns
    N = len(categories)
    angles = np.linspace(0, 2 * np.pi, N, endpoint=False).tolist()
    angles += angles[:1]  # close the loop

    plt.figure(figsize=(8, 8))
    for idx, (cluster, row) in enumerate(cluster_means.iterrows()):
        values = row.values.flatten().tolist()
        values += values[:1]  # close the loop
        plt.polar(angles, values, label=f'Cluster {cluster}', linewidth=2)

    plt.xticks(angles[:-1], categories, color='grey', size=10)
    plt.title(f'Radar Plot of Cluster Means for {best_model_name} ({group_name})', size=15, y=1.08)
    plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
    plt.tight_layout()
    plt.savefig(f"projects/proj_3_team_5/plots/radar__{best_model_name}_{group_name.lower()}.png", dpi=300)
    plt.show()

    # Add cluster labels to the original dataframe
    df_with_clusters = df_raw.copy()
    df_with_clusters['cluster'] = best_labels

    # Show five sample beers from each cluster only if number of clusters is smaller than 10
    n_clusters = len(df_with_clusters['cluster'].unique())
    if n_clusters < 10:
        print(f"\n=== Sample Beers from Each Cluster ({best_model_name}) ===")
        for cluster in sorted(df_with_clusters['cluster'].unique()):
            cluster_beers = df_with_clusters[df_with_clusters['cluster'] == cluster]
            print(f"\nCluster {cluster} ({len(cluster_beers)} beers):")
            
            # Sample 5 beers from this cluster
            sample_beers = cluster_beers.sample(n=min(5, len(cluster_beers)), random_state=42)
            display(sample_beers)
    else:
        print(f"\nSkipping sample display - too many clusters ({n_clusters})")
